In [9]:
# Tune_model.ipynb
import os

seed = 52
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

print(">>> Global env + basic seeds set, seed =", seed)
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from pathlib import Path
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold


from model_code import set_deterministic, _total_evr_from_recon, _total_r2_from_recon, QuestionnaireDataset, decorrelation_loss, batch_swap_noise
import matplotlib.pyplot as plt

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

%load_ext autoreload
%autoreload 2


device = set_deterministic(seed)

>>> Global env + basic seeds set, seed = 52
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[Seed fixed] random/numpy/torch all set to 52
Using device: cuda


In [10]:
code_dir = Path.cwd()
data_path = code_dir.parent / 'data'
data_file = data_path / 'cbcl_data_remove_low_frequency.csv'
if not data_file.exists():
    raise FileNotFoundError(f'Could not find {data_file}')

qns = pd.read_csv(data_file, encoding='utf-8')
X = qns.iloc[:, 1:].values

X_train_val, X_test = train_test_split(X, test_size=0.2, random_state=seed)


encoding_dim = 5

In [11]:
class AEModel(nn.Module):
    def __init__(self, input_dim, latent_dim, h1, h2):
        super(AEModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, h1),
            # nn.LeakyReLU(0.01),
            nn.GELU(),
            nn.Linear(h1, h2),
            # nn.LeakyReLU(0.01),
            nn.GELU(),
            # nn.Linear(h2, h3),
            # nn.LeakyReLU(0.01),
            nn.Linear(h2, latent_dim),

        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, h2),
            # nn.LeakyReLU(0.01),
            nn.GELU(),
            # nn.Linear(h3, h2),
            # nn.LeakyReLU(0.01),
            nn.Linear(h2, h1),
            # nn.LeakyReLU(0.01),
            nn.GELU(),
            nn.Linear(h1, input_dim),
            # nn.Sigmoid(),
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)


class AE:
    """
    简化版示例：Autoencoder 的外层封装
    __init__(X_train, X_val, encoding_dim, h1, h2, h3)
    train(max_epochs=..., patience=..., show_plot=...)
    evaluate_on_data(X) -> (latent, rec_errors, evr, total_evr, recon)
    """
    def __init__(self, X_train, X_val, encoding_dim, h1=0, h2=0, *,  lr, seed, lambda_decorr=None, weight_decay=None):
        train_ds = QuestionnaireDataset(X_train)
        val_ds = QuestionnaireDataset(X_val)
        g_loader = torch.Generator().manual_seed(seed)
        self.lambda_decorr = lambda_decorr
        self.train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, generator=g_loader)
        self.val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
        # self.g_noise = torch.Generator(device=device).manual_seed(seed + 1)
        self.g_noise = torch.Generator().manual_seed(seed + 1)  # CPU generator 足够

        input_dim = X_train.shape[1]
        self.model = AEModel(input_dim, encoding_dim, h1, h2).to(device)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode="min", factor=0.1, patience=5
        )

    def train(self, max_epochs=1000, patience=30, show_plot=False):
        best_val_loss = float("inf")
        epochs_no_improve = 0
        train_losses, val_losses = [], []
        for epoch in range(max_epochs):
            self.model.train()
            total_train_loss = 0
            for batch_x, _ in self.train_loader:
                batch_x = batch_x.to(device)
                x_noisy = batch_swap_noise(batch_x, swap_prob=0.1, generator=self.g_noise)
                # x_noisy = batch_x
                self.optimizer.zero_grad()
                reconstructed = self.model(x_noisy)
                rec_loss = self.criterion(reconstructed, batch_x)
                latent = self.model.encoder(batch_x)
                dec_loss = self.lambda_decorr * decorrelation_loss(latent)
                loss = rec_loss + dec_loss
                loss.backward()
                self.optimizer.step()
                total_train_loss += loss.item() * batch_x.size(0)

            train_avg = total_train_loss / len(self.train_loader.dataset)
            train_losses.append(train_avg)

            # 验证
            self.model.eval()
            total_val_loss = 0
            with torch.no_grad():
                for batch_x, _ in self.val_loader:
                    batch_x = batch_x.to(device)
                    reconstructed = self.model(batch_x)
                    rec_loss = self.criterion(reconstructed, batch_x)
                    latent = self.model.encoder(batch_x)
                    dec_loss = self.lambda_decorr * decorrelation_loss(latent)
                    total_val_loss += (rec_loss + dec_loss).item() * batch_x.size(0)

            val_avg = total_val_loss / len(self.val_loader.dataset)
            val_losses.append(val_avg)
            self.scheduler.step(val_avg)

            if val_avg < best_val_loss:
                best_val_loss = val_avg
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"Early stopping at epoch {epoch + 1}")
                    break

        if show_plot:
            plt.plot(train_losses, label="Train Loss")
            plt.plot(val_losses, label="Validation Loss")
            plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("AE Loss Curves")
            plt.legend(); plt.show()

    def evaluate_on_data(self, X):
        self.model.eval()
        X_np = np.asarray(X)
        with torch.no_grad():
            X_t = torch.tensor(X_np, dtype=torch.float32).to(device)
            recon = self.model(X_t)
            latent = self.model.encoder(X_t).cpu().numpy()
            latent_vars = np.var(latent, axis=0, ddof=1)
            total_var = np.var(X_np, axis=0, ddof=1).sum()
            evr = latent_vars / total_var
            recon_np = recon.cpu().numpy()
            rec_errors = np.mean((X_np - recon_np)**2, axis=1)
            total_evr = _total_evr_from_recon(X_np, recon_np, ddof=1)
            total_r2 = _total_r2_from_recon(X_np, recon_np)
        return latent, rec_errors, evr, total_evr, recon_np, total_r2

In [13]:
set_deterministic(seed)

X_train, X_val = train_test_split(X_train_val, test_size=0.2, random_state=seed)

scaler = StandardScaler()
# scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

autoencoder = AE(
    X_train,
    X_val,
    encoding_dim=encoding_dim,
    h1=192,
    h2=64,
    seed=seed,
    lr=0.003,
    lambda_decorr=1,
    weight_decay=8e-05,
)

autoencoder.train(show_plot=False)

latent_factors, _, _, evr_test, reconstructed, _ = autoencoder.evaluate_on_data(X_test)
print(f"Evaluating on test set: EVR test = {evr_test:.4f}")

latent_factors, _, _, evr_train, reconstructed, _ = autoencoder.evaluate_on_data(X_train)
mse_ae = float(np.mean((X_train - reconstructed) ** 2))

print(f'AE -> MSE={mse_ae:.6f}, Total EVR={evr_train:.4f}')


[Seed fixed] random/numpy/torch all set to 52
Using device: cuda
Early stopping at epoch 95
Evaluating on test set: EVR test = 0.3698
AE -> MSE=0.608519, Total EVR=0.3917


In [14]:
latent_factors, error_test, _, evr_test, reconstructed, r2_test = autoencoder.evaluate_on_data(X_test)
print(f"Evaluating on test set: EVR test = {evr_test:.6f}")
print(f"Mean reconstruction error on test set: MSE test = {np.mean(error_test):.6f}")
print(f"Total R2 on test set: R2 test = {r2_test:.6f}")

latent_factors, error_val, _, evr_val, reconstructed, r2_val = autoencoder.evaluate_on_data(scaler.transform(X_val))
print(f"Evaluating on total set: EVR total = {evr_val:.6f}")
print("Mean reconstruction error on total set:" f" MSE total = {np.mean(error_val):.6f}")
print(f"Total R2 on total set: R2 total = {r2_val:.6f}")

latent_factors, error_train, _, evr_train, reconstructed, r2_train = autoencoder.evaluate_on_data(X_train)
print(f"Evaluating on train set: EVR train = {evr_train:.6f}")
print(f"Mean reconstruction error on train set: MSE train = {np.mean(error_train):.6f}")
print(f"Total R2 on train set: R2 train = {r2_train:.6f}")  

Evaluating on test set: EVR test = 0.369844
Mean reconstruction error on test set: MSE test = 12.645459
Total R2 on test set: R2 test = 0.358717
Evaluating on total set: EVR total = 0.380717
Mean reconstruction error on total set: MSE total = 12.560674
Total R2 on total set: R2 total = 0.370054
Evaluating on train set: EVR train = 0.391696
Mean reconstruction error on train set: MSE train = 0.608519
Total R2 on train set: R2 train = 0.391481


In [15]:
set_deterministic(seed)

X_train, X_val = train_test_split(X_train_val, test_size=0.2, random_state=seed)

scaler = StandardScaler()
# scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

autoencoder = AE(
    X_train,
    X_val,
    encoding_dim=encoding_dim,
    h1=192,
    h2=64,
    seed=seed,
    lr=0.003,
    lambda_decorr=0.005,
    weight_decay=8e-05,
)

autoencoder.train(show_plot=False)

latent_factors, _, _, evr_test, reconstructed, _ = autoencoder.evaluate_on_data(X_test)
print(f"Evaluating on test set: EVR test = {evr_test:.4f}")

latent_factors, _, _, evr_train, reconstructed, _ = autoencoder.evaluate_on_data(X_train)
mse_ae = float(np.mean((X_train - reconstructed) ** 2))

print(f'AE -> MSE={mse_ae:.6f}, Total EVR={evr_train:.4f}')


[Seed fixed] random/numpy/torch all set to 52
Using device: cuda
Early stopping at epoch 82
Evaluating on test set: EVR test = 0.5343
AE -> MSE=0.581805, Total EVR=0.4184


In [16]:
latent_factors, error_test, _, evr_test, reconstructed, r2_test = autoencoder.evaluate_on_data(X_test)
print(f"Evaluating on test set: EVR test = {evr_test:.6f}")
print(f"Mean reconstruction error on test set: MSE test = {np.mean(error_test):.6f}")
print(f"Total R2 on test set: R2 test = {r2_test:.6f}")

latent_factors, error_val, _, evr_val, reconstructed, r2_val = autoencoder.evaluate_on_data(scaler.transform(X_val))
print(f"Evaluating on total set: EVR total = {evr_val:.6f}")
print("Mean reconstruction error on total set:" f" MSE total = {np.mean(error_val):.6f}")
print(f"Total R2 on total set: R2 total = {r2_val:.6f}")

latent_factors, error_train, _, evr_train, reconstructed, r2_train = autoencoder.evaluate_on_data(X_train)
print(f"Evaluating on train set: EVR train = {evr_train:.6f}")
print(f"Mean reconstruction error on train set: MSE train = {np.mean(error_train):.6f}")
print(f"Total R2 on train set: R2 train = {r2_train:.6f}")  

Evaluating on test set: EVR test = 0.534276
Mean reconstruction error on test set: MSE test = 825.946489
Total R2 on test set: R2 test = 0.523288
Evaluating on total set: EVR total = 0.489109
Mean reconstruction error on total set: MSE total = 10.468645
Total R2 on total set: R2 total = 0.474974
Evaluating on train set: EVR train = 0.418368
Mean reconstruction error on train set: MSE train = 0.581805
Total R2 on train set: R2 train = 0.418195


In [ ]:
import optuna
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler

# ======== 超参范围（全局） ========
H1_MIN, H1_MAX, H1_STEP = 64, 256, 32
H2_MIN, H2_MAX, H2_STEP = 32, 128, 16
LR_MIN, LR_MAX = 1e-4, 5e-3
LDEC_MIN, LDEC_MAX = 0.0, 1.0
WD_MIN, WD_MAX = 1e-6, 1e-3

encoding_dim = 5
outer_kf = KFold(n_splits=2, shuffle=True, random_state=seed)

# Optuna samplers
tpe_sampler = optuna.samplers.TPESampler(seed=seed)
cma_sampler = optuna.samplers.CmaEsSampler(
    seed=seed,
    sigma0=0.5,
    warn_independent_sampling=False,
)

all_results = []
fold_metadata = []
outer_fold_id = 1

# 可选：结果保存目录
nested_root = code_dir / "output" / "nested_cv_ae_optuna"
nested_root.mkdir(parents=True, exist_ok=True)

def _clip_int_range(center, lo, hi, step):
    """小工具：给定中心值，在 [lo, hi] 范围里取一个较窄区间"""
    low = max(lo, center - step)
    high = min(hi, center + step)
    # 保险起见，保证 low < high
    if low >= high:
        low = lo
        high = hi
    return low, high

for train_val_idx, test_idx in outer_kf.split(X):

    print(f"\n================ Outer Fold {outer_fold_id} ================")

    # ---------- 外层 Train/Val/Test 划分 ----------
    X_train_val_raw = X[train_val_idx]
    X_test_raw = X[test_idx]

    scaler = StandardScaler()
    X_train_val = scaler.fit_transform(X_train_val_raw)
    X_test = scaler.transform(X_test_raw)

    # ---------- inner objective（共用） ----------
    def make_objective(X_outer_train_val, search_space_mode="wide", base_params=None):

        def objective(trial):
            # ========== 1) 定义搜索空间 ==========
            if search_space_mode == "wide":
                # Stage 1：TPE 使用的“大范围”
                h1 = trial.suggest_int("h1", H1_MIN, H1_MAX, step=H1_STEP)
                h2 = trial.suggest_int("h2", H2_MIN, H2_MAX, step=H2_STEP)
                lr = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
                lambda_decorr = trial.suggest_float("lambda_decorr", LDEC_MIN, LDEC_MAX)
                weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)

            elif search_space_mode == "narrow":
                # Stage 2：CMA-ES 使用的“缩小范围”
                assert base_params is not None, "base_params 不能为空（narrow 模式）"

                h1_center = int(base_params["h1"])
                h2_center = int(base_params["h2"])
                lr_center = float(base_params["lr"])
                ldec_center = float(base_params["lambda_decorr"])
                wd_center = float(base_params["weight_decay"])

                # around h1, h2（±一个 step）
                h1_low, h1_high = _clip_int_range(h1_center, H1_MIN, H1_MAX, H1_STEP)
                h2_low, h2_high = _clip_int_range(h2_center, H2_MIN, H2_MAX, H2_STEP)

                h1 = trial.suggest_int("h1", h1_low, h1_high, step=H1_STEP)
                h2 = trial.suggest_int("h2", h2_low, h2_high, step=H2_STEP)

                # lr：以 center 为中点、对数尺度缩窄一圈
                lr_low = max(LR_MIN, lr_center / 3.0)
                lr_high = min(LR_MAX, lr_center * 3.0)
                lr = trial.suggest_float("lr", lr_low, lr_high, log=True)

                # lambda_decorr：在 [center-0.3, center+0.3] 之间截断
                ldec_low = max(LDEC_MIN, ldec_center - 0.3)
                ldec_high = min(LDEC_MAX, ldec_center + 0.3)
                if ldec_low >= ldec_high:
                    ldec_low, ldec_high = LDEC_MIN, LDEC_MAX
                lambda_decorr = trial.suggest_float("lambda_decorr", ldec_low, ldec_high)

                # weight_decay：同样 log 缩窄
                wd_low = max(WD_MIN, wd_center / 5.0)
                wd_high = min(WD_MAX, wd_center * 5.0)
                weight_decay = trial.suggest_float("weight_decay", wd_low, wd_high, log=True)

            else:
                raise ValueError(f"Unknown search_space_mode: {search_space_mode}")

            # ========== 2) inner KFold ==========
            inner_kf = KFold(n_splits=2, shuffle=True, random_state=seed)
            ev_totals = []

            inner_fold_id = 1
            for inner_train_idx, inner_val_idx in inner_kf.split(X_outer_train_val):
                print(f"-------- Inner Fold {inner_fold_id} ({search_space_mode}) --------")

                X_inner_train = X_outer_train_val[inner_train_idx]
                X_inner_val = X_outer_train_val[inner_val_idx]

                set_deterministic(seed + inner_fold_id)

                ae = AE(
                    X_inner_train,
                    X_inner_val,
                    encoding_dim=encoding_dim,
                    h1=h1,
                    h2=h2,
                    lr=lr,
                    seed=seed + inner_fold_id,
                    lambda_decorr=lambda_decorr,
                    weight_decay=weight_decay,
                )

                print(f"[Inner Fold {inner_fold_id}] Training AE...")
                ae.train(show_plot=False)

                _, _, _, ev_total_val, _, _ = ae.evaluate_on_data(X_inner_val)
                ev_totals.append(ev_total_val)
                print(f"[Inner Fold {inner_fold_id}] EV_total = {ev_total_val:.4f}")

                inner_fold_id += 1

            mean_ev = float(np.mean(ev_totals))
            print(f"[Inner Summary | mode={search_space_mode}] Mean EV_total = {mean_ev:.4f}")
            return mean_ev

        return objective

    # ---------- Stage 1：TPE 粗搜索 ----------
    objective_wide = make_objective(X_train_val, search_space_mode="wide")

    study_tpe = optuna.create_study(
        direction="maximize",
        sampler=tpe_sampler,
        study_name=f"ae_outer{outer_fold_id}_tpe",
    )
    study_tpe.optimize(objective_wide, n_trials=20)

    best_params_tpe = study_tpe.best_params
    best_value_tpe = study_tpe.best_value
    print(f"[Outer Fold {outer_fold_id}] TPE Best Params: {best_params_tpe}")
    print(f"[Outer Fold {outer_fold_id}] TPE Best EV_total = {best_value_tpe:.4f}")

    # ---------- Stage 2：CMA-ES 精细搜索 ----------
    objective_narrow = make_objective(
        X_train_val,
        search_space_mode="narrow",
        base_params=best_params_tpe,
    )

    study_cma = optuna.create_study(
        direction="maximize",
        sampler=cma_sampler,
        study_name=f"ae_outer{outer_fold_id}_cma",
    )
    study_cma.optimize(objective_narrow, n_trials=30)

    best_params_cma = study_cma.best_params
    best_value_cma = study_cma.best_value
    print(f"[Outer Fold {outer_fold_id}] CMA-ES Best Params: {best_params_cma}")
    print(f"[Outer Fold {outer_fold_id}] CMA-ES Best EV_total = {best_value_cma:.4f}")

    # 你也可以选 TPE 和 CMA-ES 里更大的那个，这里先用 CMA-ES 的结果：
    final_best_params = best_params_cma
    final_best_params["stage1_best_ev"] = best_value_tpe
    final_best_params["stage2_best_ev"] = best_value_cma

    best_h1 = final_best_params["h1"]
    best_h2 = final_best_params["h2"]
    best_lr = final_best_params["lr"]
    best_lambda_decorr = final_best_params["lambda_decorr"]
    best_weight_decay = final_best_params["weight_decay"]

    # ---------- outer fold 上用最佳超参重新训练 ----------
    X_train, X_val = train_test_split(
        X_train_val,
        test_size=0.16,   # 0.64 / 0.16 / 0.20 结构
        random_state=seed,
    )

    autoencoder = AE(
        X_train,
        X_val,
        encoding_dim=encoding_dim,
        h1=best_h1,
        h2=best_h2,
        lr=best_lr,
        seed=seed,
        lambda_decorr=best_lambda_decorr,
        weight_decay=best_weight_decay,
    )

    print(f"[Outer Fold {outer_fold_id}] Training Final AE (with best params)...")
    autoencoder.train(show_plot=False)

    # ---------- 测试集评估 ----------
    latent_factors, rec_errors, evr, ev_total, reconstructed, r2 = \
        autoencoder.evaluate_on_data(X_test)

    rec_mean = rec_errors.mean()
    print(
        f"[Outer Fold {outer_fold_id}] Final Test: "
        f"EV_total={ev_total:.4f}, RecErr={rec_mean:.4f}, R²={r2:.4f}"
    )

    all_results.append({
        "fold": outer_fold_id,
        "latent": latent_factors,
        "ev_total": ev_total,
        "reconstructed": reconstructed,
        "r2": r2,
        "rec_error": rec_errors,
    })

    fold_metadata.append({
        "fold": outer_fold_id,
        "ev_total": float(ev_total),
        "rec_error_mean": float(rec_mean),
        "r2": float(r2),
        "best_h1": int(best_h1),
        "best_h2": int(best_h2),
        "best_lr": float(best_lr),
        "best_lambda_decorr": float(best_lambda_decorr),
        "best_weight_decay": float(best_weight_decay),
        "tpe_best_ev": float(best_value_tpe),
        "cma_best_ev": float(best_value_cma),
    })

    # 顺手把每个 outer fold 对应的 study 的 trials 存一下（可选）
    study_tpe.trials_dataframe().to_csv(
        nested_root / f"outer{outer_fold_id}_tpe_trials.csv",
        index=False,
    )
    study_cma.trials_dataframe().to_csv(
        nested_root / f"outer{outer_fold_id}_cma_trials.csv",
        index=False,
    )

    outer_fold_id += 1

# ---------- 最终汇总 ----------
ev_list = [r["ev_total"] for r in all_results]
r2_list = [r["r2"] for r in all_results]
rec_list = [r["rec_error"].mean() for r in all_results]

print("\n================ Nested CV Summary ================")
print(f"EV_total mean={np.mean(ev_list):.4f}, std={np.std(ev_list):.4f}")
print(f"RecErr mean={np.mean(rec_list):.4f}, std={np.std(rec_list):.4f}")
print(f"R² mean={np.mean(r2_list):.4f}, std={np.std(r2_list):.4f}")

# ---------- 保存一个总结果（你问“该去哪儿”） ----------
import json
import pandas as pd

summary = {
    "seed": int(seed),
    "encoding_dim": int(encoding_dim),
    "outer_folds": fold_metadata,
    "summary_metrics": {
        "ev_total_mean": float(np.mean(ev_list)),
        "ev_total_std": float(np.std(ev_list)),
        "rec_err_mean": float(np.mean(rec_list)),
        "rec_err_std": float(np.std(rec_list)),
        "r2_mean": float(np.mean(r2_list)),
        "r2_std": float(np.std(r2_list)),
    },
}

# JSON：详细信息
with open(nested_root / "nested_cv_summary.json", "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

# CSV：每个 outer fold 一行，便于以后快速看表
pd.DataFrame(fold_metadata).to_csv(
    nested_root / "nested_cv_folds.csv", index=False
)

print(f"Nested CV summary saved to: {nested_root}")


Selecting from 5 folds

In [5]:
nested_root = code_dir / "output" / "nested_cv_ae_optuna"
nested_root

WindowsPath('g:/ABCD/script/trail/notebooks/output/nested_cv_ae_optuna')

In [8]:
fold_result = pd.DataFrame()
for n in range(1, 6):
    fold_ = pd.read_csv(nested_root / f"fold_{n}" / "nested_cv_folds.csv")
    fold_result = pd.concat([fold_result, fold_], axis=0)
    print(f"fold_{n}")
fold_result

fold_1
fold_2
fold_3
fold_4
fold_5


,fold,ev_total,rec_error_mean,r2,best_h1,best_h2,best_lr,best_lambda_decorr,best_weight_decay,tpe_best_ev,cma_best_ev
0,1,0.382993,0.640465,0.382556,224,96,0.002195,0.092998,0.000124,0.353428,0.355562
0,2,0.362788,0.624998,0.362262,128,64,0.003469,0.006983,0.000086,0.358757,0.355001
0,3,0.339258,0.654328,0.338698,192,80,0.003245,0.929482,0.000035,0.350982,0.352103
0,4,0.359715,0.692212,0.359167,192,64,0.002361,0.003367,0.000150,0.356611,0.361090
0,5,0.360503,0.598198,0.359983,96,48,0.003768,0.004493,0.000088,0.353440,0.357725
